# Ingeniería de características

Se crean características basadas en las originales que provean información adicional acerca de los encuestados.

## Objetivo

- Crear nuevas características que mejoren la comprensión por parte de los modelos de inteligencia artificial

- Seleccionar características originales que puedan ser usadas para el entrenamiento del modelo de inteligencia artificial

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import cv2
import json
import seaborn as sns
import os
import math
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/Shared drives/Tecnicas de aprendizaje estadistico/Trabajo 1")

Mounted at /content/drive


## Lectura de datos

Se realiza una lectura de los datos previamente procesados y organizados

In [2]:
df = pd.read_csv('datasets/formatted_data.csv')

EmptyDataError: ignored

In [ ]:
# Abrir archivo JSON con los metadatos

with open('datasets/metadata.json', 'r') as outfile:
  metadata =  json.loads(json.load(outfile))

## Creación de nuevas características

### **Caracteristicas 1, 2, 3 y 4** 

**1) tiene_servicios_basicos**: Esta caracteristica sirve para identificar si una persona cuenta con todos los servicios basicos (Energia, acueducto, alcantarillado y recolección de basuras). Se crea basandose en las correspondientes columnas tiene_energia, tiene_acueducto, tiene_alcantarillado, tiene_recoleccion_basuras. En caso de que se tengan todos estos servicios, esta variable sera ***True***, pero, si no se cuenta con uno de ellos la variable sera ***False***.

**2) calidad_sector**: Esta caracteristica sirve para identificar la calidad del sector en el que vive una persona y se comparte para todos los miembros del hogar. Aqui se analizan factores como ruidos exteriores, malos olores, presencia de basuras etc. La variable se construye sumando las respuestas dadas por las personas a los factores anteriormente mencionados. Estas respuestas van desde 1 hasta 4, siendo 1 nunca y 4 siempre. La variable se construye como:
~~~
Si 5 <= suma_de(factores del sector) <= 12, entonces calidad_sector es 1 o buena

Sino, si 13 <= suma_de(factores del sector) <= 25, entonces calidad_sector es 2 o regular

Sino, entonces calidad_sector es 3 o mala
~~~

**3) calidad_vivienda**: Esta caracteristica sirve para identificar la calidad del vivienda en el que vive una persona y se comparte para todos los miembros del hogar. Aqui se analizan factores como pintura soplada, presencia de manchas,  si se tienen grietas, etc. La variable se construye sumando las respuestas dadas por las personas a los factores anteriormente mencionados. Estas respuestas son de verdadero o false. La variable se construye como:

~~~
Si presenta grietas graves, entonces calidad_vivienda es 3 o mala

Sino

  Si 0 <= suma_de(factores de vivienda) <= 2, entonces calidad_vivienda es 1 o buena

  Sino, si 3 <= suma_de(factores de vivienda) <= 4, entonces calidad_sector es 2 o regular

  Sino, entonces calidad_vivienda es 3 o mala
~~~

**4) cantidad_personas_hogar**: Esta caracteristica sirve para identificar la cantidad de personas en un hogar y es igual para todos los miembros de ese hogar. Simplemente se construye buscando la respuesta del primer miembro del hogar y asociandola a todos los que no la tengan. 

In [ ]:
"""Codigo para creación de las caracteristicas 1, 2, 3 y 4"""

lista_servicios = []
lista_sectores = []
lista_vivienda = []
lista_cantidad_personas = []

grouped_by_vivienda = df.groupby("hogar")

for name, group in grouped_by_vivienda:

    vivienda_row = group.loc[(group["id_visita"] == 1) & (group["id_persona_hogar"] == 1)]

    """ Caracteristica: Servicios publicos basicos """
    if vivienda_row["tiene_energia"].values[0] and vivienda_row["tiene_acueducto"].values[0] and vivienda_row["tiene_alcantarillado"].values[0] and vivienda_row["tiene_recoleccion_basuras"].values[0]: 
      tiene_servicios = True
    else:
      tiene_servicios = False
    lista_servicios += [tiene_servicios] * len(group)

    """ Caracteristica: Calidad del sector """
    cantidad_problemas_sector = float(vivienda_row["ruidos_exteriores"].values[0]) + float(vivienda_row["malos_olores"].values[0]) + float(vivienda_row["presencia_basuras"].values[0]) + float(vivienda_row["contaminacion_aire"].values[0]) + float(vivienda_row["contaminacion_rios"].values[0]) + float(vivienda_row["invasion_espacio_publico"].values[0]) + float(vivienda_row["animales_molestos"].values[0]) + float(vivienda_row["insectos_molestos"].values[0])

    # La calidad 3 del sector corresponde a un sector malo, calidad 2 a un sector regular y calidad 1 a un sector bueno.
    if 5 <= cantidad_problemas_sector <= 12:
      calidad_sector = 1
    elif 13 <= cantidad_problemas_sector <= 25:
      calidad_sector = 2
    else:
      calidad_sector = 3
    lista_sectores += [calidad_sector] * len(group)

    """ Caracteristica: Calidad de la vivienda """
    presenta_grietas_graves = bool(vivienda_row["grietas_graves"].values[0])

    # La calidad 3 de la vivienda corresponde a una vivienda malo, calidad 2 a una vivienda regular y calidad 1 a una vivienda buena.
    if presenta_grietas_graves:
      calidad_vivienda = 3
    else:
      cantidad_problemas_vivienda = float(vivienda_row["pintura_soplada"].values[0]) + float(vivienda_row["presencia_manchas"].values[0]) + float(vivienda_row["presencia_moho"].values[0]) + float(vivienda_row["cambios_aspecto_ladrillos"].values[0]) +  float(vivienda_row["goteras"].values[0]) + float(vivienda_row["grietas_leves"].values[0])
      if 0 <= cantidad_problemas_vivienda <= 2:
        calidad_vivienda = 1
      elif 3 <= cantidad_problemas_vivienda <= 4:
        calidad_vivienda = 2
      else:
        calidad_vivienda = 3
    lista_vivienda += [calidad_vivienda] * len(group)

    """ Caracteristica: Cantidad de personas en el hogar """
    lista_cantidad_personas += [int(vivienda_row["cantidad_personas_hogar"].values[0])] * len(group)

# Asignación de las variables a las columnas correspondientes
df["tiene_servicios_basicos"] = lista_servicios
df["calidad_sector"] = lista_sectores
df["calidad_vivienda"] = lista_vivienda
df["cantidad_personas_hogar"] = lista_cantidad_personas

### **Caracteristica 5**

**5) actualmente_trabaja:** Se construye esta variable para ver si la persona actualmente trabaja. Esta variable se construye a partir de la duración actual de una persona en un trabajo. Si la persona lleva mas de 0 meses trabajando en una empresa, industria, negocio o finca, es porque se encuentra trabajando. Si no responde o lleva 0 meses, es porque actualmente no trabaja. 

In [ ]:
condiciones = [(df['meses_trabajo'] > 0), 
               (df['meses_trabajo'] == 0) |  (df['meses_trabajo'].isnull())]
valores = [True, False]
df["actualmente_trabaja"] = np.select(condiciones, valores)

### **Caracteristica 6** 

**6) ingresos_mensuales:** Se construye esta variable para conocer los ingresos de las personas sumando su salario y lo que obtienen por pensiones, jubilacion, etc. Esta variable es interesante para las personas mayores de 60 años o abuelos. 

In [ ]:
df["ingresos_mensuales"] = df["salario"] + df["pension"]

### **Caracteristicas 7, 8, 9 y 10** 


**7 y 8) padre_hogar, madre_hogar:** Se construyen esta variables para saber si en el caso de los niños, los padres viven o no en su mismo hogar. Si se encuentra que el niño registra un orden padre y orden madre, entonces se sabe que ambos viven en el hogar.

**9 y 10) padre_vive, madre_vive:** Se construyen esta variables para saber si en el caso de los niños, los padres se encuentran con vidas. Si se encuentra que el niño respondio si o no a la pregunta de si el padre y la madren viven en el hogar, entonces se asume que el padre y la madre estan vivos. Si se respondio que esta fallecido, entonces el padre o la madre de este niño estan fallecidos.



In [ ]:
# 1 es True - 0 es False
df["padre_hogar"] = ~df['orden_padre'].isnull()
df["madre_hogar"] = ~df['orden_madre'].isnull()

# 1 y 2 son True - 3 es False
df["padre_vive"] = ~(df["padre_vive_en_residencia"].isnull())
df["madre_vive"] = ~(df["madre_vive_en_residencia"].isnull())

### **Transformación de variable objetivo en niños** 
Debido a que se miro en diferentes bases de datos y los niños no daban respuesta a la pregunta de la satisfacción de vida, se decide sustituir sus respuestas por el promedio de las respuesta de las personas que comparten su vivienda. Ademas, pueden haber personas adultas (No abuelos) que tampoco hayan dado respuesta a esta pregunta, por lo que tambien se les reemplazara por el promedio de las personas de la vivienda.

In [ ]:
mean_satisfaccion_vivienda = df.groupby('hogar')["satisfaccion_vida"].mean()
df['satisfaccion_vida'] = df.apply(lambda row: mean_satisfaccion_vivienda[row['hogar']] if pd.isnull(row['satisfaccion_vida']) else row['satisfaccion_vida'], axis=1) 
df.dropna(subset=['satisfaccion_vida'], inplace=True)
df['satisfaccion_vida'] = df.apply(lambda row: math.floor(row['satisfaccion_vida']) if math.modf(row['satisfaccion_vida'])[0] <= 0.5 else math.ceil(row['satisfaccion_vida']), axis=1) 

### Guardar dataframe y metadata

In [ ]:
import json
base = 'datasets'
with open(base + '/metadata.json', 'w') as outfile:
    json.dump(json.dumps(metadata), outfile)

In [ ]:
dataframe_name_to_export = "formatted_data.csv"
base = 'datasets'
df.to_csv(base + "/" + dataframe_name_to_export, index=False)